In [ ]:
# Módulos de Astropy
from astropy.io import fits
from astropy.table import Table

# Módulos de visualización y cálculo numérico
import matplotlib.pyplot as plt
import numpy as np

# Módulos de sklearn para división de datos y búsqueda en grilla
from sklearn.model_selection import train_test_split, GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

# Módulos de TensorFlow y TensorFlow Addons
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Activation, LeakyReLU
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import ModelCheckpoint, LambdaCallback

# Módulo de itertools para la generación de productos cartesianos
from itertools import product

path_datos = "../data/"
path = path_datos + "ZW2.msw_image.fits"
hdul = fits.open(path)

path_cat = "catalogo.cat"
cat = Table.read(path_cat, format="ascii")

# Crear un archivo de anotación
clase = "fuente"
with open("galaxias_sextractor_marcos.txt", "w") as f:
    for row in cat:
        # Obtener las coordenadas de la galaxia
        x = row["X_IMAGE"]
        y = row["Y_IMAGE"]

        # Escribir la anotación en el archivo
        f.write(f"{x} {y} {clase}\n")

# Leer el archivo de texto con las posiciones de las galaxias
with open("galaxias_sextractor_marcos.txt", "r") as f:
    lines = f.readlines()

# Crear un ndarray de ceros con la misma forma que tus imágenes
label = np.zeros((2048, 2048), dtype=int)

# Función para mantener las coordenadas dentro de los límites de la imagen
def clamp(value, min_value, max_value):
    return max(min(value, max_value), min_value)

# Establecer a 1 las posiciones correspondientes a las galaxias
for line in lines:
    x, y, _ = line.split()
    x, y = int(float(x)), int(float(y))

    # Establecer 1 en un área de 8x8 píxeles alrededor de la posición central de cada galaxia
    for i in range(-4, 4):
        for j in range(-4, 4):
            x_coord = clamp(x + i, 0, 2047)
            y_coord = clamp(y + j, 0, 2047)
            label[y_coord, x_coord] = 1


all_data = hdul[0].data

# Seleccionar x frecuencias distintas de manera aleatoria
num_samples = 400
indices = sorted(np.random.choice(all_data.shape[0], num_samples, replace=False))
data = all_data[indices]
print(indices)

# Guardar los índices en un archivo de texto
def save_indices_to_txt(indices, filename="indices.txt"):
    np.savetxt(filename, indices, fmt='%d')

save_indices_to_txt(indices, "indices.txt")


# Reformatear y normalizar las imágenes
input_data = np.array(data)
input_data = input_data.reshape((data.shape[0], data.shape[1], data.shape[2], 1)) #600, 2048, 2048, 1
input_data = input_data.astype("float32") / 255.0

# Crear un array de salida repetido x veces (uno para cada imagen de entrada)
output_data = np.repeat(label[np.newaxis, :, :], data.shape[0], axis=0)

# Dividir los datos en conjuntos de entrenamiento y validación
x_train, x_val, y_train, y_val = train_test_split(input_data, output_data, test_size=0.2, random_state=42)


def plot_history(history, title="", show=True):
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))

    # Gráfico de precisión
    axes[0].plot(history.history["accuracy"], label="Entrenamiento")
    axes[0].plot(history.history["val_accuracy"], label="Validación")
    axes[0].set_xlabel("Épocas")
    axes[0].set_ylabel("Precisión")
    axes[0].set_title("Precisión - " + title)
    axes[0].legend()

    # Gráfico de pérdida
    axes[1].plot(history.history["loss"], label="Entrenamiento")
    axes[1].plot(history.history["val_loss"], label="Validación")
    axes[1].set_xlabel("Épocas")
    axes[1].set_ylabel("Pérdida")
    axes[1].set_title("Pérdida - " + title)
    axes[1].legend()

    if show:
        plt.show()
    

    # Construir la CNN
model = Sequential()
model.add(Conv2D(32, (3, 3), activation="relu", padding="same", input_shape=(data.shape[1], data.shape[2], 1)))
model.add(tfa.layers.InstanceNormalization())
model.add(MaxPooling2D((2, 2), padding="same"))
model.add(Conv2D(32, (3, 3), activation="relu", padding="same"))
model.add(tfa.layers.InstanceNormalization())
model.add(MaxPooling2D((2, 2), padding="same"))
model.add(Conv2D(32, (3, 3), activation="relu", padding="same"))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation="relu", padding="same"))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(1, (3, 3), activation="sigmoid", padding="same"))

# Compilar el modelo
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()


# Entrenar el modelo
history = model.fit(x_train, y_train, epochs=5, batch_size=2, validation_data=(x_val, y_val))
model.save("cnn1.h5")
plot_history(history)

In [2]:
import tensorflow as tf

gpus = tf.config.list_physical_devices("GPU")
if gpus:
    try:
        # Si hay varias GPU, selecciona solo la primera (cambia el índice si deseas utilizar otra GPU)
        tf.config.experimental.set_visible_devices(gpus[0], "GPU")
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
else:
    print("No se encontró GPU")


No se encontró GPU


2023-04-19 16:49:45.954738: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-19 16:49:45.954900: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [1]:
# Módulos de Astropy
from astropy.io import fits
from astropy.table import Table

# Módulos de visualización y cálculo numérico
import matplotlib.pyplot as plt
import numpy as np

# Módulos de sklearn para división de datos
from sklearn.model_selection import train_test_split

# Módulos de PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Módulos de Astropy
from astropy.io import fits
from astropy.table import Table

path_datos = "../data/"
path = path_datos + "ZW2.msw_image.fits"
hdul = fits.open(path)

path_cat = "catalogo.cat"
cat = Table.read(path_cat, format="ascii")

# Crear un archivo de anotación
clase = "fuente"
with open("galaxias_sextractor_marcos.txt", "w") as f:
    for row in cat:
        # Obtener las coordenadas de la galaxia
        x = row["X_IMAGE"]
        y = row["Y_IMAGE"]

        # Escribir la anotación en el archivo
        f.write(f"{x} {y} {clase}\n")

# Leer el archivo de texto con las posiciones de las galaxias
with open("galaxias_sextractor_marcos.txt", "r") as f:
    lines = f.readlines()

# Crear un ndarray de ceros con la misma forma que tus imágenes
label = np.zeros((2048, 2048), dtype=int)

# Función para mantener las coordenadas dentro de los límites de la imagen
def clamp(value, min_value, max_value):
    return max(min(value, max_value), min_value)

# Establecer a 1 las posiciones correspondientes a las galaxias
for line in lines:
    x, y, _ = line.split()
    x, y = int(float(x)), int(float(y))

    # Establecer 1 en un área de 8x8 píxeles alrededor de la posición central de cada galaxia
    for i in range(-4, 4):
        for j in range(-4, 4):
            x_coord = clamp(x + i, 0, 2047)
            y_coord = clamp(y + j, 0, 2047)
            label[y_coord, x_coord] = 1

all_data = hdul[0].data

# Seleccionar x frecuencias distintas de manera aleatoria
num_samples = 400
indices = sorted(np.random.choice(all_data.shape[0], num_samples, replace=False))
data = all_data[indices]

# Guardar los índices en un archivo de texto
def save_indices_to_txt(indices, filename="indices.txt"):
    np.savetxt(filename, indices, fmt='%d')

save_indices_to_txt(indices, "indices.txt")

# Reformatear y normalizar las imágenes
input_data = np.array(data)
input_data = input_data.reshape((data.shape[0], data.shape[1], data.shape[2], 1)) # 400, 2048, 2048, 1
input_data = input_data.astype("float32") / 255.0



In [3]:
# Crear un array de salida repetido x veces (uno para cada imagen de entrada)
output_data = np.repeat(label[np.newaxis, :, :], data.shape[0], axis=0)

# Dividir los datos en conjuntos de entrenamiento y validación
x_train, x_val, y_train, y_val = train_test_split(input_data, output_data, test_size=0.2, random_state=42)

# Convertir los datos a tensores de PyTorch
x_train = torch.tensor(x_train).float()
x_val = torch.tensor(x_val).float()
y_train = torch.tensor(y_train).float()
y_val = torch.tensor(y_val).float()

# Definir la arquitectura de la red neuronal
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, padding=1),
            nn.Conv2d(32, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, padding=1)
        )
        self.decoder = nn.Sequential(
            nn.Conv2d(32, 32, 3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(32, 32, 3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(32, 1, 3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Crear la CNN, la función de pérdida y el optimizador
model = CNN().cuda()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

# Entrenar el modelo
num_epochs = 5
batch_size = 2
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for i in range(0, len(x_train), batch_size):
        inputs = x_train[i:i+batch_size].cuda()
        labels = y_train[i:i+batch_size].cuda()

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    train_loss /= len(x_train) / batch_size
    train_losses.append(train_loss)

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for i in range(0, len(x_val), batch_size):
            inputs = x_val[i:i+batch_size].cuda()
            labels = y_val[i:i+batch_size].cuda()

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
    val_loss /= len(x_val) / batch_size
    val_losses.append(val_loss)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

# Guardar el modelo
torch.save(model.state_dict(), "cnn_pytorch.pt")

# Gráfico de pérdida
plt.plot(train_losses, label="Entrenamiento")
plt.plot(val_losses, label)


RuntimeError: Given groups=1, weight of size [32, 1, 3, 3], expected input[2, 2048, 2048, 1] to have 1 channels, but got 2048 channels instead